# Dependences

In [33]:
include("../../src/struct_data.jl")
include("../../src/neighbor.jl")

dist_kernel! (generic function with 1 method)

# Input Data

## Model Parameter

In [34]:
model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 10000.0,
        dt    = 0.5,
        nₖₙₙ  = 50,
        nₛₐᵥₑ = 100
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input  = "../../data/init/Sphere"
    ),
    OutputModel(
        name_output = "Test_1",
        path_output = ".results/cont_par(A)/force_par(B)/",
        d_saved = 0.5
    ) 
)
dump(Model)

## Initial Aggregate

In [35]:
FusionAGG = Aggregate(
    [AggType(
        "HEK_1", 
        InteractionPar( Cubic(0.2,2.0,3.0), ContractilePar(0.0005,pi/4,0.08,1.0)),
        Float32.(readdlm("../../data/init/Sphere/$(15).0.xyz")[3:end,2:end]) |> cu
    )], 
    [AggLocation("HEK_1",[0 0 0]),],
    model
)
show_aggregates(FusionAGG)

1-element Vector{AggType}:
 AggType("HEK_1", InteractionPar(Cubic{Float64}(0.2, 2.0, 3.0), ContractilePar{Float64}(0.0005, 0.7853981633974483, 0.08, 1.0)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})

1×2504 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

1×2504 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

1×2504 Matrix{String}:
 "HEK_1"  "HEK_1"  "HEK_1"  "HEK_1"  …  "HEK_1"  "HEK_1"  "HEK_1"  "HEK_1"

2504×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -1.5  -4.62  -13.88
  0.5  -4.62  -13.88
  2.5  -4.62  -13.88
 -4.5  -2.89  -13.88
 -2.5  -2.89  -13.88
 -0.5  -2.89  -13.88
  1.5  -2.89  -13.88
  3.5  -2.89  -13.88
 -5.5  -1.15  -13.88
 -3.5  -1.15  -13.88
  ⋮           
  4.5   1.15   13.88
 -4.5   2.89   13.88
 -2.5   2.89   13.88
 -0.5   2.89   13.88
  1.5   2.89   13.88
  3.5   2.89   13.88
 -1.5   4.62   13.88
  0.5   4.62   13.88
  2.5   4.62   13.88

1×2504 Matrix{Float32}:
 15.27  15.27  15.27  15.27  15.27  …  15.27  15.27  15.27  15.27  15.27

1×2504 Matrix{Int64}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

Cubic{CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}}(Float32[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2  …  0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2], Float32[2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0  …  2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0], Float32[3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0  …  3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0])

ContractilePar{CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}}(Float32[0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005  …  0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005, 0.0005], Float32[0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982  …  0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982, 0.7853982], Float32[0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08  …  0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08], Float32[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

1-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 15.27

# Program

In [38]:
FusionAGG.Simulation.Neighbor.idx_red

1000×2504 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 ⋮              ⋮              ⋮        ⋱           ⋮              ⋮        
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  

In [49]:
display("------------------------ IDX -------------------------")
display(FusionAGG.Simulation.Neighbor.idx_red)
display(FusionAGG.Simulation.Neighbor.idx_sum)

display("------------------------ SIZE ------------------------")
display("Size X         = $(size(FusionAGG.Position))")
display("Size idx_sum   = $(size(FusionAGG.Simulation.Neighbor.idx_sum))")
display("Size idx       = $(size(FusionAGG.Simulation.Neighbor.idx_red))")

"------------------------ IDX -------------------------"

1000×2504 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 ⋮              ⋮              ⋮        ⋱           ⋮              ⋮        
 0  0  0  0  0  0  0  0  0  0  0  0  0     0  0  0  0  0  0  0  0  0  0  0  0
 0  0  0  0  

1×2504 CuArray{Int32, 2, CUDA.Mem.DeviceBuffer}:
 0  0  0  0  0  0  0  0  0  0  0  0  0  …  0  0  0  0  0  0  0  0  0  0  0  0

"------------------------ SIZE ------------------------"

"Size X         = (2504, 3)"

"Size idx_sum   = (1, 2504)"

"Size idx       = (1000, 2504)"